# E4: Using a Clinician-Approved Subset of Labeling Functions
It has been observed that our set of 57 labeling functions are likely redundant and overly dependent, to the extent that model performance is suffering. So we asked Chris to give us a subset of 10-15 of our labeling functions that he finds "most important" for our task. The following code implements a Label Model using this subset and evaluates the new model.

The list of LFs in the subset are on the "Implements LFs" tab at the following link: https://docs.google.com/spreadsheets/d/1_1QBVaiWl4SkBy9vEFBk5Uv0HfPWUMX8Sg_IPZoJbfA/edit?usp=sharing

In [ ]:
# Extract relevant parts of label matrix

lf_subset = [57, 58, 59, 60, 61, 17, 23, 30, 41, 52]

L_data = np.copy(L_alarms[:,lf_subset])

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from snorkel.labeling.model.label_model import LabelModel

all_scores = []
all_cmtx = np.zeros((3,3))
metrics = ["accuracy","coverage","precision","recall","f1","roc_auc"]
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

for i, (train_idx, test_idx) in enumerate(kf.split(L_data)):
    # Define training dataset
    L_train = L_data[train_idx]
    Y_train = alarms_df.true_label.values[train_idx]
    # Define test dataset
    L_test = L_data[test_idx]
    Y_test = alarms_df.true_label.values[test_idx]

    # Fit a label model
    l_model = LabelModel(cardinality=2, verbose=True)
    l_model.fit(L_train, n_epochs=100, log_freq=10, seed=SEED)

    # Evaluate
    scores = l_model.score(L_test, Y=Y_test, metrics=metrics, tie_break_policy="abstain")
    Y_pred = l_model.predict(L_test, tie_break_policy="abstain")
    conf_mtx = confusion_matrix(Y_test, Y_pred)

    all_cmtx = all_cmtx + conf_mtx
    all_scores.append(scores)

    print("-- ITERATION ", i+1, " --")
    print(conf_mtx)
    print("Abstain: ", np.sum(Y_pred == ABSTAIN) / len(Y_pred))
    print("Supress: ", np.sum(Y_pred == SUPPRESSIBLE) / len(Y_pred))
    print("Scores: ", scores, '\n')

In [ ]:
# Summarized scores from summing confusion matrices over iterations

print("-- SUMMARY --")
print(all_cmtx)
print("Abstain: ", np.sum(Y_pred == ABSTAIN) / len(Y_pred))
print("Supress: ", np.sum(Y_pred == SUPPRESSIBLE) / len(Y_pred))
print("accuracy: ", (all_cmtx[1,1] + all_cmtx[2,2]) / np.sum(all_cmtx[1:,1:]))
print("coverage: ", np.sum(all_cmtx[1:,1:]) / np.sum(all_cmtx))
print("precision: ", all_cmtx[2,2] / np.sum(all_cmtx[1:,2]))
print("recall: ", all_cmtx[2,2] / np.sum(all_cmtx[2,1:]))
print("f1: ", all_cmtx[2,2] / (all_cmtx[2,2] + 0.5 *(all_cmtx[1,2] + all_cmtx[2,1])))